# 🧬 Pipeline di Integrazione Dati Genici "Semantic-Aware" (SNF + MOFA+)

Questa pipeline è progettata per l'integrazione di dati funzionali (Gene Ontology, HPO) al fine di identificare cluster di geni funzionalmente correlati. Adotta una duplice strategia, combinando la topologia locale di **SNF (Similarity Network Fusion)** con la scomposizione della varianza globale di **MOFA+ (Multi-Omics Factor Analysis)**, culminando in una proiezione UMAP guidata per un clustering robusto e semanticamente ricco.

---

## 1. Fase: Pre-processing "Semantic-Aware" 🧹

**Obiettivo:** Ridurre il rumore e la ridondanza semantica nelle matrici funzionali, mantenendo la pertinenza biologica.

### 1.1. Filtraggio Iniziale per Frequenza
* **Termini Rari:** Eliminare i termini funzionali associati a meno di **3 geni** (troppo rari).
* **Termini Generici:** Eliminare i termini associati a più del **20%** del totale dei geni (troppo generici).

### 1.2. Semantic Redundancy Removal
Si sfrutta la struttura del **Grafo Aciclico Diretto (DAG)** di GO/HPO per calcolare la **Similarità Semantica** (es. Wang o Resnik).

**Algoritmo:** Se $Sim(t_1, t_2) > 0.7$ **E** $t_1$ è antenato di $t_2$ (più generale), allora **scartare $t_1$** (mantenendo il termine $t_2$, che è più specifico/informativo).

### 1.3. Gestione Geni Orfani
Rimuovere i geni che, dopo il filtraggio dei termini, risultano avere vettori $[0,0,...,0]$ in tutte le viste.

---

## 2. Fase: Feature Engineering & Trasformazione ⚙️

**Obiettivo:** Preparare le matrici di input (Geni $\times$ Termini) per le strategie parallele SNF e MOFA.

### 2.1. Calcolo TF-IDF
Applicare la metrica **Term Frequency-Inverse Document Frequency (TF-IDF)** alle matrici binarie iniziali.

$$W_{i,j} = tf_{i,j} \times \log\left(\frac{N}{df_j}\right)$$

### 2.2. Creazione delle Matrici di Input

* **Per MOFA+:** Mantenere le matrici **TF-IDF** (Gene $\times$ Termini) come input diretto ("Views").
* **Per SNF:** Calcolare le matrici di **Similarità** (Gene $\times$ Gene) usando **Cosine Similarity** sulle matrici TF-IDF.

$$S_{view} = \frac{X_{tfidf} \cdot X_{tfidf}^T}{||X_{tfidf}|| \cdot ||X_{tfidf}||^T}$$

---

## 3. Fase: Dual Integration Strategy (SNF & MOFA+) 🤝

### Strada A: Similarity Network Fusion (SNF) - Topologia Locale
**Obiettivo:** Fondere le similarità amplificando i legami forti.
* **Gestione HPO:** Includere $S_{HPO}$. SNF gestisce la sparsità penalizzando naturalmente le viste rumorose.
* **Parametri:** $K$ dinamico (es. 20), $t=20$ iterazioni.
* **Output A:** Matrice fusa $W_{fused}$ (Gene $\times$ Gene). 

### Strada B: MOFA+ (Multi-Omics Factor Analysis) - Varianza Globale
**Obiettivo:** Estrarre fattori latenti che spiegano la variabilità.
* **Input:** Le 4 matrici TF-IDF (CC, MF, BP, HPO) come Views.
* **Processo:** Modellazione Bayesiana per fattorizzare le matrici: $Y_{view} = Z \cdot W_{view}^T + \epsilon$.
* **Output B:** Matrice dei Fattori Latenti $Z_{MOFA}$ (Geni $\times$ Fattori), es. 5183 $\times$ 15.

---

## 4. Fase: Integrazione Dimensionale & Proiezione (UMAP Guidato) 🗺️

**Obiettivo:** Unire la topologia locale di SNF con la spiegabilità globale di MOFA per la proiezione finale.

Si usa **UMAP** con un'inizializzazione guidata:
* **Metrica:** Usare la distanza derivata da SNF ($D = 1 - W_{fused}$) per definire la topologia locale.
* **Inizializzazione:** Usare i primi $N$ fattori di MOFA ($Z_{MOFA}$) come punto di partenza per l'embedding, guidando la struttura globale.

---

## 5. Fase: Clustering Robusto (Soft Clustering) 📊

**Obiettivo:** Identificare gruppi di geni funzionalmente correlati senza forzare il rumore.

### 5.1. HDBSCAN

Applicare HDBSCAN all'embedding UMAP generato
* **Parametri:** min_cluster_size=30, min_samples=10.

### 5.2. Recupero Rumore (Soft Clustering)
Generare vettori di probabilità per ogni gene ($P(gene \in Cluster_i)$).

**Regola:** Se un gene è classificato come **-1 (rumore)** ma ha una probabilità di appartenenza ($\lambda$) per un Cluster X **maggiore di $0.3$**, riassegnalo al Cluster X.

---

## 6. Fase: Validazione & Explainability (Biologica) 🧪

### A. Enrichment Analysis (Validazione Esterna)

Per ogni cluster, eseguire test ipergeometrici su **KEGG/Reactome** (FDR < 0.05) per convalidare la coerenza funzionale.

### B. MOFA Factor Characterization (Validazione Interna)
Usare i pesi dei fattori di MOFA ($W_{view}$) per capire i driver della separazione:

* Analizzare quali viste (BP, CC, MF, HPO) pesano di più sui Fattori Latenti che separano i cluster.
* **Esempio:** "Il Cluster 1 e 2 sono separati lungo il Fattore 1, che è guidato al 80% dalla vista HPO (termini legati a dismorfismi)."


---

#### Imports:

In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import umap
import hdbscan
from matplotlib.lines import Line2D
import urllib.request
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from snf import snf

c:\Users\husse\Desktop\Uni\ScientificVisualization\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def calculate_tfidf_and_similarity(df, view_name):
    print(f"\n--- Elaborazione Vista: {view_name} ---")
    n_genes = df.shape[0]
    
    doc_freq = df.sum(axis=0) 
    
    idf = np.log(n_genes / doc_freq + 1e-10)
    
    print("   Calcolo similarità (può richiedere qualche secondo)...")
    
    weighted_matrix = df.values * np.sqrt(idf.values)
    
    weighted_intersection = np.dot(weighted_matrix, weighted_matrix.T)
    
    gene_sums = (df * idf).sum(axis=1).values
    
    weighted_union = gene_sums[:, None] + gene_sums[None, :] - weighted_intersection
    
    with np.errstate(divide='ignore', invalid='ignore'):
        similarity = weighted_intersection / weighted_union
        similarity[np.isnan(similarity)] = 0.0
    
    np.fill_diagonal(similarity, 1.0)
    
    sim_df = pd.DataFrame(similarity, index=df.index, columns=df.index)
    
    print(f"   Matrice completata: {sim_df.shape}")
    return sim_df

input_files = {
    "BP": "filtered_BP.csv",
    "CC": "filtered_CC.csv",
    "MF": "filtered_MF.csv",
    "HPO": "filtered_HPO.csv" 
}

similarity_results = {}

for key, filename in input_files.items():
    if os.path.exists(filename):
        # Carica dati
        df = pd.read_csv(filename, index_col=0)
        
        # Calcola
        sim_matrix = calculate_tfidf_and_similarity(df, key)
        
        # Salva
        output_file = f"similarity_{key}.csv"
        sim_matrix.to_csv(output_file)
        print(f"   -> Salvato in: {output_file}")
        
        similarity_results[key] = sim_matrix
    else:
        print(f"ATTENZIONE: File {filename} non trovato. Hai eseguito la Fase 1?")

print("\nFase 2 Completata.")

In [ ]:
def analyze_view(sim_matrix_path, view_name):
    print(f"\n--- Analisi Vista: {view_name} ---")
    
    sim_df = pd.read_csv(sim_matrix_path, index_col=0)
    
    distance_matrix = 1 - sim_df.values
    distance_matrix[distance_matrix < 0] = 0
    
    reducer = umap.UMAP(
        n_neighbors=30,
        min_dist=0.1,
        n_components=2,
        metric='precomputed',
        random_state=42
    )
    
    embedding = reducer.fit_transform(distance_matrix)
    
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=30,
        metric='euclidean',
        cluster_selection_method='eom'
    )
    
    cluster_labels = clusterer.fit_predict(embedding)
    
    n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
    n_noise = list(cluster_labels).count(-1)
    print(f"   -> Trovati {n_clusters} cluster.")
    print(f"   -> Geni scartati come rumore: {n_noise}")

    plt.figure(figsize=(10, 8))
    
    # Definizione colori
    noise_color = (0.8, 0.8, 0.8)
    palette = sns.color_palette('tab20', n_colors=n_clusters)
    cluster_colors = [palette[x] if x >= 0 else noise_color for x in cluster_labels]
    
    plt.scatter(
        embedding[:, 0], 
        embedding[:, 1], 
        c=cluster_colors, 
        s=5, 
        alpha=0.6
    )
    
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Noise', 
               markerfacecolor=noise_color, markersize=10),
    ]
    plt.legend(handles=legend_elements, loc='upper right')

    plt.title(f'UMAP Projection - {view_name} ({n_clusters} clusters)', fontsize=16)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    
    plt.savefig(f"plot_umap_{view_name}.png", dpi=300)
    plt.close()
    
    results = pd.DataFrame({
        'Gene': sim_df.index,
        'Cluster': cluster_labels,
        'UMAP_1': embedding[:, 0],
        'UMAP_2': embedding[:, 1]
    })
    results.to_csv(f"clusters_{view_name}.csv", index=False)
    
    return n_clusters, n_noise

files = {
    "BP": "similarity_BP.csv",
    "CC": "similarity_CC.csv",
    "MF": "similarity_MF.csv",
    "HPO": "similarity_HPO.csv"
}

summary = []
for name, path in files.items():
    n_clust, n_noise = analyze_view(path, name)
    summary.append({'View': name, 'Clusters': n_clust, 'Noise_Genes': n_noise})

print(pd.DataFrame(summary))

### STARTING CODE

In [ ]:
obo_files = {
    "GO": {"url": "https://current.geneontology.org/ontology/go-basic.obo", "path": "go-basic.obo"},
    "HPO": {"url": "https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.obo", "path": "hp.obo"}
}

data_files = {
    "BP": r"C:\Users\husse\Desktop\Uni\ScientificVisualization\csv_data\gene_go_matrix_propT_rel-is_a-part_of_ont-BP.csv",
    "CC": r"C:\Users\husse\Desktop\Uni\ScientificVisualization\csv_data\gene_go_matrix_propT_rel-is_a-part_of_ont-CC.csv",
    "MF": r"C:\Users\husse\Desktop\Uni\ScientificVisualization\csv_data\gene_go_matrix_propT_rel-is_a-part_of_ont-MF.csv",
    "HPO": r"C:\Users\husse\Desktop\Uni\ScientificVisualization\csv_data\gene_hpo_matrix_binary_withAncestors_namespace_Phenotypic_abnormality.csv"
}

depth_files = {
    "BP": "./csv_data/goterm_depth_propT_rel-is_a-part_of_ont-BP.csv",
    "CC": "./csv_data/goterm_depth_propT_rel-is_a-part_of_ont-CC.csv",
    "MF": "./csv_data/goterm_depth_propT_rel-is_a-part_of_ont-MF.csv"
}

def check_and_download(url, filename):
    if not os.path.exists(filename):
        try:
            urllib.request.urlretrieve(url, filename)
        except Exception as e:
            print(e)

for key, info in obo_files.items():
    check_and_download(info["url"], info["path"])


def load_ontology(obo_path):
    try:
        return GODag(obo_path)
    except Exception as e:
        print(e)
        return None

def normalize_id(term_id):
    term_id = str(term_id)
    if "GO" in term_id or "HP" in term_id:
        return term_id.replace(".", ":").replace("_", ":")
    return term_id

def filter_by_depth(df, depth_file, min_depth=4):
    
    try:
        df_depth = pd.read_csv(depth_file, index_col=0, nrows=1)
        depth_series = pd.to_numeric(df_depth.iloc[0], errors='coerce').dropna()
        valid_terms = set(depth_series[depth_series >= min_depth].index)
        
        cols_to_keep = []
        for col in df.columns:
            if col in valid_terms or normalize_id(col) in valid_terms:
                cols_to_keep.append(col)
                
        return df[cols_to_keep]
    except Exception as e:
        print(e)
        return df

def frequency_filtering(df, min_genes=3, max_pct=0.20):
    counts = df.sum(axis=0)
    limit = df.shape[0] * max_pct
    mask = (counts >= min_genes) & (counts <= limit)
    df_filtered = df.loc[:, mask]
    return df_filtered

def remove_semantic_redundancy(df, dag, threshold=0.7):

    example_col = df.columns[0]
    normalized_ex = normalize_id(example_col)
    if normalized_ex not in dag:
        print(f"Il termine '{example_col}' (norm: '{normalized_ex}') non è stato trovato nel DAG!")


    matrix = (df.values > 0).astype(int).T
    intersect = matrix @ matrix.T
    row_sums = matrix.sum(axis=1)
    union = row_sums[:, None] + row_sums[None, :] - intersect
    
    with np.errstate(divide='ignore', invalid='ignore'):
        sim_matrix = np.triu(intersect / union, k=1)

    pairs = np.where(sim_matrix >= threshold)
    to_drop = set()
    cols = df.columns
    
    match_count = 0
    
    for i, j in zip(*pairs):
        term_a_raw = cols[i]
        term_b_raw = cols[j]
        
        term_a = normalize_id(term_a_raw)
        term_b = normalize_id(term_b_raw)
        
        if term_a in dag and term_b in dag:
            match_count += 1
            parents_b = dag[term_b].get_all_parents()
            parents_a = dag[term_a].get_all_parents()
            
            if term_a in parents_b:
                to_drop.add(term_a_raw)
            elif term_b in parents_a:
                to_drop.add(term_b_raw)
            
    return df.drop(columns=list(to_drop))

def keep_common_active_genes(dfs_dict):
    print("--- Filtro Geni Comuni Attivi ---")
    
    # 1. Trova i geni che hanno almeno un valore != 0 in OGNI vista
    valid_genes_per_view = []
    
    for name, df in dfs_dict.items():
        # Calcola la somma per riga (gene)
        row_sums = df.sum(axis=1)
        # Tieni solo i geni con somma > 0
        active_genes = set(row_sums[row_sums > 0].index)
        valid_genes_per_view.append(active_genes)
        print(f"   Vista {name}: {len(active_genes)} geni attivi su {len(df)}")

    common_genes = set.intersection(*valid_genes_per_view)
    common_genes = sorted(list(common_genes))
    
    print(f"   -> Geni validi comuni rimasti: {len(common_genes)}")
    
    filtered_dict = {}
    for name, df in dfs_dict.items():
        # .loc seleziona solo le righe dei geni comuni
        filtered_dict[name] = df.loc[common_genes]
        
    return filtered_dict

go_dag = load_ontology(obo_files["GO"]["path"])
hpo_dag = load_ontology(obo_files["HPO"]["path"])

processed_dfs = {}

for key, path in data_files.items():
    try:
        df = pd.read_csv(path, index_col=0)
        
        if key in depth_files:
            df = filter_by_depth(df, depth_files[key], min_depth=4)
        df = frequency_filtering(df)
        
        current_dag = hpo_dag if key == "HPO" else go_dag
        
        df = remove_semantic_redundancy(df, current_dag, threshold=0.7)
        
        processed_dfs[key] = df
        
    except FileNotFoundError:
        print(f"File non trovato: {path}")

if processed_dfs:
    final_dfs = keep_common_active_genes(processed_dfs)
    print("\n=== Salvataggio ===")
    for key, df in final_dfs.items():
        out_name = f"filtered_final_{key}.csv"
        df.to_csv(out_name)
        print(f"   -> {key}: {df.shape} salvato in {out_name}")

In [ ]:
input_files = {
    "BP": "filtered_final_BP.csv",
    "CC": "filtered_final_CC.csv",
    "MF": "filtered_final_MF.csv",
    "HPO": "filtered_final_HPO.csv"
}

def process_phase2(file_path, view_name):
    print(f"\n--- Elaborazione Vista: {view_name} ---")
    
    # 1. Caricamento Dati (Matrice Binaria)
    try:
        df_binary = pd.read_csv(file_path, index_col=0)
        print(f"   Input (Binario): {df_binary.shape}")
    except FileNotFoundError:
        print(f"   ERRORE: File {file_path} non trovato. Salto.")
        return

    tfidf_transformer = TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True)
    
    tfidf_matrix_sparse = tfidf_transformer.fit_transform(df_binary)
    
    df_tfidf = pd.DataFrame(
        tfidf_matrix_sparse.toarray(), 
        index=df_binary.index, 
        columns=df_binary.columns
    )
    print(f"   TF-IDF Calcolato. Range valori: [{df_tfidf.values.min():.4f}, {df_tfidf.values.max():.4f}]")

    # 3. Output per MOFA (Gene x Termini, Pesato)
    mofa_filename = f"mofa_input_{view_name}.csv"
    df_tfidf.to_csv(mofa_filename)

    cosine_sim = cosine_similarity(tfidf_matrix_sparse)
    
    # Ricostruzione DataFrame simmetrico Gene x Gene
    df_sim = pd.DataFrame(
        cosine_sim,
        index=df_binary.index,
        columns=df_binary.index
    )
    
    # Check integrità (Diagonale deve essere 1.0)
    diag_mean = np.diag(df_sim).mean()
    print(f"   Similarità Calcolata: {df_sim.shape}. Media diagonale: {diag_mean:.2f} (atteso 1.0)")

    snf_filename = f"snf_similarity_{view_name}.csv"
    df_sim.to_csv(snf_filename)

for key, filename in input_files.items():
    process_phase2(filename, key)

In [ ]:
def analyze_view(sim_matrix_path, view_name):
    print(f"\n--- Analisi Vista: {view_name} ---")
    
    sim_df = pd.read_csv(sim_matrix_path, index_col=0)
    
    distance_matrix = 1 - sim_df.values
    distance_matrix[distance_matrix < 0] = 0
    
    reducer = umap.UMAP(
        n_neighbors=30,
        min_dist=0.1,
        n_components=2,
        metric='precomputed',
        random_state=42
    )
    
    embedding = reducer.fit_transform(distance_matrix)
    
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=30,
        metric='euclidean',
        cluster_selection_method='eom'
    )
    
    cluster_labels = clusterer.fit_predict(embedding)
    
    n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
    n_noise = list(cluster_labels).count(-1)
    print(f"   -> Trovati {n_clusters} cluster.")
    print(f"   -> Geni scartati come rumore: {n_noise}")

    plt.figure(figsize=(10, 8))
    
    # Definizione colori
    noise_color = (0.8, 0.8, 0.8)
    palette = sns.color_palette('tab20', n_colors=n_clusters)
    cluster_colors = [palette[x] if x >= 0 else noise_color for x in cluster_labels]
    
    plt.scatter(
        embedding[:, 0], 
        embedding[:, 1], 
        c=cluster_colors, 
        s=5, 
        alpha=0.6
    )
    
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Noise', 
               markerfacecolor=noise_color, markersize=10),
    ]
    plt.legend(handles=legend_elements, loc='upper right')

    plt.title(f'UMAP Projection - {view_name} ({n_clusters} clusters)', fontsize=16)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    
    plt.savefig(f"plot_umap_{view_name}.png", dpi=300)
    plt.close()
    
    results = pd.DataFrame({
        'Gene': sim_df.index,
        'Cluster': cluster_labels,
        'UMAP_1': embedding[:, 0],
        'UMAP_2': embedding[:, 1]
    })
    results.to_csv(f"clusters_{view_name}.csv", index=False)
    
    return n_clusters, n_noise

In [ ]:
files = {
    "BP": "snf_similarity_BP.csv",
    "CC": "snf_similarity_CC.csv",
    "MF": "snf_similarity_MF.csv",
    "HPO": "snf_similarity_HPO.csv"
}

summary = []
for name, path in files.items():
    n_clust, n_noise = analyze_view(path, name)
    summary.append({'View': name, 'Clusters': n_clust, 'Noise_Genes': n_noise})

print(pd.DataFrame(summary))

In [ ]:
def run_snf_integration(view_names, output_dir='.'):
    """
    Carica le matrici di similarità e le fonde usando Similarity Network Fusion (SNF).
    """
    print("--- Fase 3: Similarity Network Fusion (SNF) ---")
    
    similarity_matrices = []
    gene_list = None
    
    # 1. Caricamento e Verifica delle Matrici
    for name in view_names:
        filename = f"snf_similarity_{name}.csv"
        file_path = os.path.join(output_dir, filename)
        
        if not os.path.exists(file_path):
            print(f"ERRORE: File {filename} non trovato. Esegui la Fase 2 (Cella 13).")
            return
        
        df = pd.read_csv(file_path, index_col=0)
        
        # Inizializza l'elenco dei geni con la prima vista
        if gene_list is None:
            gene_list = df.index.tolist()
        
        # Verifica che gli indici dei geni siano coerenti tra le viste
        if df.index.tolist() != gene_list:
             # Questo non dovrebbe succedere se la Fase 1 è corretta, ma è una buona prassi
            print(f"ATTENZIONE: Indici dei geni non consistenti per {name}. Ignoro.")
            continue
            
        similarity_matrices.append(df.values)
        print(f"   Caricato {name}: {df.shape}")

    if not similarity_matrices:
        print("Nessuna matrice di similarità valida trovata. SNF annullato.")
        return

    # 2. Esecuzione SNF
    # Parametri raccomandati: K=numero di vicini, t=numero di iterazioni
    K = 20  # Numero di vicini nel grafo (tipico: 10-30)
    t = 20  # Numero di iterazioni (tipico: 10-50)
    
    print(f"   Esecuzione SNF con K={K}, t={t} su {len(similarity_matrices)} matrici...")
    
    # snf.snf accetta una lista di array numpy
    W_fused = snf(similarity_matrices, K=K, t=t)
    
    print("   SNF completato.")
    
    # 3. Salvataggio della Matrice Fusa
    df_fused = pd.DataFrame(W_fused, index=gene_list, columns=gene_list)
    output_file = os.path.join(output_dir, "snf_fused_similarity_matrix.csv")
    df_fused.to_csv(output_file)
    
    print(f"   Matrice Fusa (W_fused) salvata: {df_fused.shape} in {output_file}")
    
    return df_fused

In [ ]:
view_names = ["BP", "CC", "MF", "HPO"]
df_fused_snf = run_snf_integration(view_names)

if df_fused_snf is not None:
    # Controlla la distribuzione dei valori fusi
    print(f"\nRange Matrice Fusa: [{df_fused_snf.values.min():.4f}, {df_fused_snf.values.max():.4f}]")

In [14]:
def simulate_mofa_factors(output_dir='.'):
    print("\n--- Fase 3: Simulazione MOFA+ Factors (Z_MOFA) ---")
    
    # Tentativo di caricare un file di input per ottenere l'elenco dei geni
    try:
        df_bp = pd.read_csv(os.path.join(output_dir, "mofa_input_BP.csv"), index_col=0)
        gene_list = df_bp.index.tolist()
        N_GENES = len(gene_list)
        print(f"   Caricati {N_GENES} geni validi.")
    except FileNotFoundError:
        print("ERRORE: I file 'mofa_input_BP.csv' non sono stati trovati (Eseguire Cella 13).")
        return None

    # Numero di Fattori Latenti MOFA che desideriamo estrarre (da Fase 3: 15)
    N_FACTORS = 15
    
    # SIMULAZIONE: Crea una matrice (N_GENES x N_FACTORS) con valori casuali normalizzati.
    # Questo simula l'output di un modello MOFA addestrato.
    np.random.seed(42) # Per riproducibilità
    # Genera valori casuali distribuiti normalmente
    mofa_factors_data = np.random.randn(N_GENES, N_FACTORS)
    
    # Normalizzazione per avere una distribuzione sensata
    mofa_factors_data = (mofa_factors_data - mofa_factors_data.min()) / (mofa_factors_data.max() - mofa_factors_data.min())
    mofa_factors_data = 2 * mofa_factors_data - 1 # Scala tra -1 e 1
    
    # Crea il DataFrame
    factor_names = [f"Factor_{i+1}" for i in range(N_FACTORS)]
    df_mofa_factors = pd.DataFrame(
        mofa_factors_data, 
        index=gene_list, 
        columns=factor_names
    )
    
    # Salvataggio dell'Output B
    output_file = os.path.join(output_dir, "mofa_latent_factors.csv")
    df_mofa_factors.to_csv(output_file)
    
    print(f"   Simulazione completata. Matrice Z_MOFA ({N_GENES}x{N_FACTORS}) salvata in {output_file}")
    
    return df_mofa_factors

# Esecuzione della simulazione
simulate_mofa_factors()


--- Fase 3: Simulazione MOFA+ Factors (Z_MOFA) ---
   Caricati 3317 geni validi.
   Simulazione completata. Matrice Z_MOFA (3317x15) salvata in .\mofa_latent_factors.csv


,Factor_1,Factor_2,Factor_3,Factor_4,Factor_5,Factor_6,Factor_7,Factor_8,Factor_9,Factor_10,Factor_11,Factor_12,Factor_13,Factor_14,Factor_15
16,0.109556,-0.032422,0.143314,0.339037,-0.053863,-0.053859,0.351599,0.170089,-0.106480,0.119807,-0.105126,-0.105643,0.052595,-0.429310,-0.387192
18,-0.127233,-0.227972,0.068757,-0.204538,-0.317293,0.326207,-0.051990,0.013592,-0.320076,-0.123229,0.023295,-0.258865,0.082498,-0.135808,-0.066729
19,-0.136047,0.412656,-0.004525,-0.238007,0.182411,-0.274483,0.045194,-0.439682,-0.298485,0.042510,0.163611,0.036810,-0.027366,-0.068833,-0.332099
20,-0.162462,-0.104504,0.234862,0.075325,-0.395717,0.070957,-0.087610,-0.152864,0.135262,0.229021,0.206724,-0.189153,-0.070646,0.072562,0.216621
21,-0.108649,-0.043020,-0.248880,-0.268975,0.180171,0.301743,-0.017608,0.222879,0.079353,-0.145754,0.079300,0.342392,-0.009518,0.348342,-0.587273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729230,-0.161074,0.104320,-0.304503,-0.055763,0.089172,0.028122,-0.165878,0.162847,0.057185,-0.468599,0.100300,-0.095909,-0.118887,-0.187075,-0.225780
790955,0.044684,0.120582,0.032742,0.210787,0.128493,-0.450257,-0.206438,0.264219,-0.285337,-0.613049,0.407999,0.064814,-0.154236,-0.191206,0.139226
100127206,0.105372,0.071654,-0.043711,-0.287841,-0.009549,0.236656,-0.122685,0.132890,-0.091981,0.177648,-0.220620,0.001553,0.100559,-0.152535,0.079572
100134444,-0.008115,-0.011380,-0.153050,0.221495,-0.212835,0.061047,-0.191387,-0.016303,0.142235,-0.066093,-0.689333,-0.302176,-0.484028,0.013677,-0.002628


In [5]:
## SNF e MOFA+ integrati

def analyze_dual_integration(sim_matrix_snf_path, mofa_factors_path):
    print(f"\n--- Fase 4/5: UMAP DUAL (SNF Metric + MOFA Init) ---")
    
    # 1. Caricamento Dati
    try:
        # Carica Matrice di Similarità Fusa SNF (per la Metrica)
        sim_df = pd.read_csv(sim_matrix_snf_path, index_col=0)
        distance_matrix = 1 - sim_df.values
        distance_matrix[distance_matrix < 0] = 0 # Assicura distanze non negative
        
        # Carica Matrice Fattori Latenti MOFA (per l'Inizializzazione)
        df_mofa_factors = pd.read_csv(mofa_factors_path, index_col=0)
        
        # Allinea gli indici e seleziona i Fattori per l'inizializzazione (es. i primi 15)
        # Assicurati che i geni siano nello stesso ordine!
        initial_embedding = df_mofa_factors.loc[sim_df.index].values
        
    except FileNotFoundError as e:
        print(f"ERRORE di caricamento: {e}. Esegui le celle precedenti.")
        return None, None
    except Exception as e:
        print(f"ERRORE di allineamento/selezione: {e}")
        return None, None
    
    # 2. Riduzione Dimensionale (UMAP)
    N_COMPONENTS = 2
    initial_embedding = df_mofa_factors.loc[sim_df.index].iloc[:, :N_COMPONENTS].values
    
    # 2. Riduzione Dimensionale (UMAP)
    reducer = umap.UMAP(
        n_neighbors=30,
        min_dist=0.1,
        n_components=N_COMPONENTS,
        metric='precomputed', 
        init=initial_embedding, # Ora di shape (N_GENES, 2)
        random_state=42
    )
    
    print("   Esecuzione UMAP (SNF-Guided by MOFA)...")
    # L'input è la matrice di distanza SNF. L'inizializzazione è Z_MOFA.
    embedding = reducer.fit_transform(distance_matrix)
    
    # 3. Clustering Robusto (HDBSCAN)
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=30,
        metric='euclidean', 
        cluster_selection_method='eom'
    )
    
    print("   Esecuzione HDBSCAN...")
    cluster_labels = clusterer.fit_predict(embedding)
    
    # 4. Risultati e Statistiche
    n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
    n_noise = list(cluster_labels).count(-1)
    
    print(f"   -> Risultati Dual: Trovati {n_clusters} cluster.")
    print(f"   -> Risultati Dual: Geni scartati come rumore: {n_noise} ({n_noise/len(cluster_labels)*100:.2f}%)")

    # 5. Salvataggio
    results = pd.DataFrame({
        'Gene': sim_df.index,
        'Cluster': cluster_labels,
        'UMAP_1': embedding[:, 0],
        'UMAP_2': embedding[:, 1]
    })
    results.to_csv("clusters_dual_fused_mofa.csv", index=False)
    print("   Risultati salvati in clusters_dual_fused_mofa.csv")
    
    # La visualizzazione del plot è omessa qui per brevità, ma puoi riaggiungerla con le tue librerie.
    
    return n_clusters, n_noise

In [6]:
analyze_dual_integration(
    sim_matrix_snf_path="snf_fused_similarity_matrix.csv",
    mofa_factors_path="mofa_latent_factors.csv"
)


--- Fase 4/5: UMAP DUAL (SNF Metric + MOFA Init) ---
   Esecuzione UMAP (SNF-Guided by MOFA)...


c:\Users\husse\Desktop\Uni\ScientificVisualization\venv\lib\site-packages\umap\umap_.py:1865: UserWarning: using precomputed metric; inverse_transform will be unavailable
  warn("using precomputed metric; inverse_transform will be unavailable")
c:\Users\husse\Desktop\Uni\ScientificVisualization\venv\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


   Esecuzione HDBSCAN...
   -> Risultati Dual: Trovati 3 cluster.
   -> Risultati Dual: Geni scartati come rumore: 28 (0.84%)
   Risultati salvati in clusters_dual_fused_mofa.csv


c:\Users\husse\Desktop\Uni\ScientificVisualization\venv\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\husse\Desktop\Uni\ScientificVisualization\venv\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


(3, 28)

### ORA BISOGNA CAPIRE PERCHé i GENI SONO STATI RAGGRUPPATI IN QUESTI 3 CLUSTERS (FASE 6)

In [ ]:
## SOLO SNF

def analyze_fused_view(sim_matrix_path):
    print(f"\n--- Fase 4/5: UMAP/HDBSCAN sulla Matrice Fusa ---")
    
    # 1. Caricamento Dati
    try:
        sim_df = pd.read_csv(sim_matrix_path, index_col=0)
    except FileNotFoundError:
        print(f"ERRORE: File {sim_matrix_path} non trovato. Esegui la Cella precedente.")
        return
    
    # Conversione in Matrice di Distanza: D = 1 - S
    distance_matrix = 1 - sim_df.values
    distance_matrix[distance_matrix < 0] = 0 # Assicura che le distanze siano non negative
    
    # 2. Riduzione Dimensionale (UMAP)
    reducer = umap.UMAP(
        n_neighbors=30,
        min_dist=0.1,
        n_components=2,
        metric='precomputed', # Usa la distanza derivata da SNF
        random_state=42
    )
    
    print("   Esecuzione UMAP (precomputed)...")
    embedding = reducer.fit_transform(distance_matrix)
    
    # 3. Clustering Robusto (HDBSCAN)
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=30,
        metric='euclidean', # Metrica sull'embedding UMAP
        cluster_selection_method='eom'
    )
    
    print("   Esecuzione HDBSCAN...")
    cluster_labels = clusterer.fit_predict(embedding)
    
    # 4. Risultati e Statistiche
    n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
    n_noise = list(cluster_labels).count(-1)
    
    print(f"   -> Risultati Fusi: Trovati {n_clusters} cluster.")
    print(f"   -> Risultati Fusi: Geni scartati come rumore: {n_noise} ({n_noise/len(cluster_labels)*100:.2f}%)")

    # 5. Visualizzazione e Salvataggio
    plt.figure(figsize=(10, 8))
    
    noise_color = (0.8, 0.8, 0.8)
    palette = sns.color_palette('tab20', n_colors=n_clusters)
    cluster_colors = [palette[x] if x >= 0 else noise_color for x in cluster_labels]
    
    plt.scatter(
        embedding[:, 0], 
        embedding[:, 1], 
        c=cluster_colors, 
        s=5, 
        alpha=0.6
    )
    
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Noise', 
               markerfacecolor=noise_color, markersize=10),
    ]
    plt.legend(handles=legend_elements, loc='upper right')

    plt.title(f'UMAP Projection - SNF Fused ({n_clusters} clusters)', fontsize=16)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    
    plt.savefig("plot_umap_snf_fused.png", dpi=300)
    plt.close()
    
    results = pd.DataFrame({
        'Gene': sim_df.index,
        'Cluster': cluster_labels,
        'UMAP_1': embedding[:, 0],
        'UMAP_2': embedding[:, 1]
    })
    results.to_csv("clusters_snf_fused.csv", index=False)
    print("   Risultati e Plot salvati (clusters_snf_fused.csv e plot_umap_snf_fused.png)")
    
    return n_clusters, n_noise


In [ ]:
analyze_fused_view("snf_fused_similarity_matrix.csv")

In [11]:
import scipy.stats as stats
import pingouin as pg
# Nota: pingouin è spesso usato per Kruskal-Wallis

# 1. Caricamento Dati
df_clusters = pd.read_csv("clusters_dual_fused_mofa.csv")
df_factors = pd.read_csv("mofa_latent_factors.csv", index_col=0) # Contiene 10 colonne (F1 a F10)

# Allineamento e rimozione del rumore (-1)
df_merged = pd.merge(df_clusters, df_factors, left_on='Gene', right_index=True)
df_analysis = df_merged[df_merged['Cluster'] != -1].copy()

significant_factors = []

# 2. Loop per il Test Statistico (usando Kruskal-Wallis, più robusto)
for i in range(1, 11): # Loop sui 10 fattori
    factor_col = f'Factor_{i}'
    
    # Esegue il test Kruskal-Wallis
    
    aov = pg.kruskal(data=df_analysis, dv=factor_col, between='Cluster')
    p_value = aov['p-unc'].iloc[0]
        
    if p_value < 0.001:
        significant_factors.append({
            'Factor': factor_col,
            'p_value': p_value,
            'Status': 'Highly Significant'
        })


In [13]:
print(significant_factors)

[]
